<a href="https://colab.research.google.com/github/viniciusrpb/116319_estruturasdedados/blob/main/cap9_4_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from sklearn.metrics import classification_report
import numpy as np



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "/content/drive/My Drive/leafs" "leafs"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255.)

train_set = train_datagen.flow_from_directory("leafs/train",
                                              subset='training',
                                              target_size=(224,224),
                                              class_mode="categorical",
                                              batch_size=32)

validation_set = train_datagen.flow_from_directory("leafs/train",
                                              subset='validation',
                                              target_size=(224,224),
                                              class_mode="categorical",
                                              batch_size=32)

test_set = test_datagen.flow_from_directory("leafs/test",
                                            target_size=(224,224),
                                            class_mode="categorical",
                                            batch_size=32)



Found 160 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [ ]:
kernel_dimensions = (5,5)
tam_strides = (1,1)
number_filters = 50
number_of_neurons = 30

num_classes = 4

Transfer Learning

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape=(224,224,3),
                                include_top=False,
                                weights='imagenet')

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable=False

In [ ]:
ans = layers.Flatten()(pre_trained_model.output)

ans = layers.Dense(512,activation="relu")(ans)

ans = layers.Dropout(0.2)(ans)

ans = layers.Dense(num_classes,activation="softmax")(ans)

model_tl = Model(pre_trained_model.input,ans)

In [ ]:
#model_tl.summary()

In [ ]:
model_tl.compile(optimizer=SGD(learning_rate=0.1),
                 loss = "categorical_crossentropy",
                 metrics=["accuracy"])

In [ ]:
history_fine = model_tl.fit(train_set,
                            epochs=20,
                            validation_data=validation_set
                            #initial_epoch=history.epoch[-1],
                            )

Epoch 1/20
5/5 [==============================] - 49s 3s/step - loss: 798.6488 - accuracy: 0.3000 - val_loss: 5129.9814 - val_accuracy: 0.2500
Epoch 2/20
5/5 [==============================] - 12s 3s/step - loss: 1266.4167 - accuracy: 0.3438 - val_loss: 3.6887 - val_accuracy: 0.2500
Epoch 3/20
5/5 [==============================] - 12s 3s/step - loss: 3.6896 - accuracy: 0.2875 - val_loss: 2.5255 - val_accuracy: 0.2500
Epoch 4/20
5/5 [==============================] - 12s 3s/step - loss: 1.6025 - accuracy: 0.2625 - val_loss: 1.3896 - val_accuracy: 0.2500
Epoch 5/20
5/5 [==============================] - 12s 3s/step - loss: 1.3912 - accuracy: 0.2500 - val_loss: 1.3887 - val_accuracy: 0.2500
Epoch 6/20
5/5 [==============================] - 13s 3s/step - loss: 1.3899 - accuracy: 0.2500 - val_loss: 1.3880 - val_accuracy: 0.2500
Epoch 7/20
5/5 [==============================] - 12s 3s/step - loss: 1.3893 - accuracy: 0.2500 - val_loss: 1.3874 - val_accuracy: 0.2500
Epoch 8/20
5/5 [==========

In [ ]:
y_prob = model_tl.predict(test_set)
y_pred = np.argmax(y_prob,axis=1)

print(classification_report(test_set.classes,y_pred))

              precision    recall  f1-score   support

           0       0.25      1.00      0.40        25
           1       1.00      0.04      0.08        25
           2       0.00      0.00      0.00        25
           3       0.00      0.00      0.00        25

    accuracy                           0.26       100
   macro avg       0.31      0.26      0.12       100
weighted avg       0.31      0.26      0.12       100



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CIFAR10

In [ ]:
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [ ]:
# load dataset
(trainX, trainy), (testX, testy) = cifar10.load_data()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


In [ ]:
trainX = trainX.astype('float32')
testX = testX.astype('float32')

#X_train = trainX / 255.0
#X_test = testX / 255.0
for ch in range(3):
  trainX[:,:,:,ch] = (trainX[:,:,:,ch]-trainX[:,:,:,ch].min())/(trainX[:,:,:,ch].max()-trainX[:,:,:,ch].min())
  testX[:,:,:,ch] = (testX[:,:,:,ch]-testX[:,:,:,ch].min())/(testX[:,:,:,ch].max()-testX[:,:,:,ch].min())

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(trainX, trainy, test_size=0.2, random_state=42)

#y_train = to_categorical(y_train)
#y_valid = to_categorical(y_valid)
y_test = testy#to_categorical(testy)

In [ ]:
y_valid.shape


(10000, 1)

Data Augmentation

In [ ]:
train_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1 )

val_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1)

test_generator = ImageDataGenerator()

train_generator.fit(X_train)
val_generator.fit(X_valid)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import VGG19,ResNet50

pre_trained_model = VGG19(include_top=False,weights='imagenet',input_shape=(32,32,3),classes=4)

#pre_trained_model = InceptionV3(input_shape=(32,32,3),
#                                include_top=False,
#                                weights='imagenet')

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable=False

In [ ]:
model_vgg = Sequential()
model_vgg.add(pre_trained_model) #Adds the base model (in this case vgg19 to model_1)
model_vgg.add(Flatten())
model_vgg.add(Dense(512))
model_vgg.add(Activation("relu"))
model_vgg.add(Dropout(0.2))
model_vgg.add(Dense(num_classes,activation="softmax"))

In [ ]:
model_vgg.compile(optimizer=SGD(learning_rate=0.1),
                 loss = "categorical_crossentropy",
                 metrics=["accuracy"])

In [ ]:
history_fine = model_vgg.fit(train_generator.flow(X_train,y_train,batch_size=32),
                            epochs=20,
                            validation_data=val_generator.flow(X_valid,y_valid,batch_size=32))
                            #initial_epoch=history.epoch[-1],
                            #)

Epoch 1/20
1250/1250 [==============================] - 48s 38ms/step - loss: nan - accuracy: 0.9883 - val_loss: nan - val_accuracy: 1.0000
Epoch 2/20
1250/1250 [==============================] - 47s 38ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 3/20
1250/1250 [==============================] - 47s 38ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 4/20
1250/1250 [==============================] - 47s 38ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 5/20
1250/1250 [==============================] - 47s 38ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 6/20
1250/1250 [==============================] - 48s 38ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 7/20
1250/1250 [==============================] - 48s 38ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 8/20
1250/1250

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(learning_rate=lrate)#, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=32)


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_13 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               4194816   
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)              

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 69.04%
